# # Нейро-секретарь: Telegram-бот для протоколов совещаний
#

# ## Конфигурация среды


Этот блок кода настраивает переменные окружения и подавляет предупреждения, чтобы избежать конфликтов между библиотеками и убрать лишние сообщения в логах, что делает работу бота более стабильной и чистой.

In [1]:
import os
import warnings

# Подавляем специфические предупреждения
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# ## Установка зависимостей
# Выполните эту ячейку для установки необходимых библиотек

In [2]:
!pip install -q python-telegram-bot openai yt-dlp whisper noisereduce soundfile numpy tqdm pydub nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.5/669.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.3 MB/s eta 0:00:00


**Зта установка помогает избежать несовместимостей и неожиданных изменений в API, которые могут возникнуть при обновлении до более новой версии.**

In [3]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [4]:
!sudo apt-get install -y ffmpeg > /dev/null  # Для обработки аудио

In [5]:
!pip install git+https://github.com/openai/whisper.git # Для Транскрибации аудиофайлов

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-dgi3f1dl
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-dgi3f1dl
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━

# ## Импорт библиотек и настройка окружения


In [6]:
import os
import logging
import warnings
import asyncio
from dataclasses import dataclass
from getpass import getpass

import openai
import whisper
import yt_dlp
import noisereduce as nr
import soundfile as sf
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters
from pydub import AudioSegment

# ## Конфигурация системы

In [7]:
# Конфигурация
@dataclass
class Config:
    TELEGRAM_TOKEN: str
    OPENAI_API_KEY: str
    WHISPER_MODEL: str = "base"
    GPT_MODEL: str = "gpt-4-turbo"
    AUDIO_CACHE: str = "audio_cache"
    MAX_FILE_SIZE_MB: int = 50
    TEMP_ANALYSIS: float = 0.2
    TEMP_PROTOCOL: float = 0.5
    SUPPORTED_FORMATS: tuple = ('wav', 'mp3', 'ogg', 'flac')

    def __post_init__(self):
        os.makedirs(self.AUDIO_CACHE, exist_ok=True)

# ## Обработка аудио с прогресс-баром


In [8]:
# Класс для обработки аудио
class AudioProcessor:
    def __init__(self, config: Config):
        self.config = config
        self.logger = logging.getLogger("AudioProcessor")

    async def process_input(self, update: Update, input_source: str) -> tuple:
        """
        Определяет источник ввода: аудиофайл из Telegram или ссылка на YouTube,
        скачивает аудио и конвертирует его в WAV (если требуется).
        Возвращает кортеж (путь к аудиофайлу, тип источника).
        """
        if update.message.audio:
            try:
                # Получаем объект файла и скачиваем его
                tg_file = await update.message.audio.get_file()
                file_path = os.path.join(self.config.AUDIO_CACHE, f"{tg_file.file_id}.mp3")
                await tg_file.download_to_drive(custom_path=file_path)
                source_type = "telegram"
                self.logger.info("Аудиофайл из Telegram успешно загружен.")
            except Exception as e:
                self.logger.error(f"Ошибка загрузки файла: {e}")
                raise Exception("Ошибка при загрузке аудиофайла из Telegram.")
        elif update.message.text:
            # Если текст содержит ссылку на YouTube
            if "youtube.com" in input_source or "youtu.be" in input_source:
                source_type = "youtube"
                file_path = await self.download_youtube_audio(input_source)
            else:
                raise Exception("Неподдерживаемый формат. Отправьте аудиофайл или ссылку на YouTube.")
        else:
            raise Exception("Не удалось определить источник ввода.")

        # Если файл не в WAV, выполняется конвертация
        if not file_path.lower().endswith('.wav'):
            wav_path = await self._convert_to_wav(file_path)
            try:
                os.remove(file_path)
            except Exception:
                pass  # Если не удалось удалить, просто продолжаем
            file_path = wav_path

        return file_path, source_type

    async def _convert_to_wav(self, input_path: str) -> str:
        """Конвертация аудио в формат WAV с помощью pydub"""
        try:
            output_path = os.path.splitext(input_path)[0] + ".wav"
            audio = AudioSegment.from_file(input_path)
            audio.export(output_path, format="wav")
            self.logger.info("Конвертация в WAV завершена.")
            return output_path
        except Exception as e:
            self.logger.error(f"Ошибка конвертации: {e}")
            raise Exception("Ошибка при конвертации аудио в WAV формат.")

    async def download_youtube_audio(self, url: str) -> str:
        """Скачивает аудио с YouTube с помощью yt_dlp"""
        try:
            ydl_opts = {
                'format': 'bestaudio/best',
                'outtmpl': os.path.join(self.config.AUDIO_CACHE, '%(id)s.%(ext)s'),
                'quiet': True,
                'no_warnings': True,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(url, download=True)
                file_path = ydl.prepare_filename(info)
            self.logger.info("Аудио с YouTube успешно загружено.")
            return file_path
        except Exception as e:
            self.logger.error(f"Ошибка при загрузке аудио с YouTube: {e}")
            raise Exception("Ошибка при загрузке аудио с YouTube.")

# ## Обработка текста

In [9]:
class MeetingProcessor:
    def __init__(self, config: Config):
        self.config = config
        self.whisper_model = whisper.load_model(config.WHISPER_MODEL)
        openai.api_key = config.OPENAI_API_KEY
        self.logger = logging.getLogger("MeetingProcessor")

    async def process_meeting(self, audio_path: str) -> str:
        """
        Транскрибирует аудио с помощью модели Whisper и генерирует протокол встречи с помощью GPT.
        """
        try:
            self.logger.info("Начало транскрипции аудио...")
            result = self.whisper_model.transcribe(audio_path)
            transcription = result.get("text", "")
            self.logger.info("Транскрипция завершена.")

            protocol = self.generate_protocol(transcription)
            return protocol
        except Exception as e:
            self.logger.error(f"Ошибка обработки встречи: {e}")
            raise Exception("Ошибка при обработке аудио.")

    def generate_protocol(self, transcription: str) -> str:
        """
        Отправляет транскрипцию в OpenAI GPT для генерации протокола совещания.
        """
        try:
            prompt = f"Создай протокол совещания на основе следующей транскрипции:\n\n{transcription}"
            response = openai.ChatCompletion.create(
                model=self.config.GPT_MODEL,
                messages=[
                    {"role": "system", "content": "Ты помощник, который создает протоколы совещаний."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.5,
            )
            protocol = response.choices[0].message.content.strip()
            return protocol
        except Exception as e:
            self.logger.error(f"Ошибка генерации протокола: {e}")
            raise Exception("Ошибка при генерации протокола.")


# ## Telegram Bot

In [10]:
# Основной класс Telegram-бота
class NeuroSecretaryBot:
    def __init__(self, config: Config):
        self.config = config
        self.audio_processor = AudioProcessor(config)
        self.meeting_processor = MeetingProcessor(config)
        self.app = Application.builder().token(config.TELEGRAM_TOKEN).build()

        self.app.add_handler(CommandHandler("start", self.start))
        self.app.add_handler(MessageHandler(filters.AUDIO | filters.TEXT, self.handle_input))

        logging.basicConfig(
            format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
            level=logging.INFO
        )
        self.logger = logging.getLogger("Bot")

    async def start(self, update: Update, context):
        await update.message.reply_text(
            "🤖 Нейро-секретарь готов к работе!\n\n"
            "Отправьте аудиофайл (MP3/WAV/OOG) или ссылку на YouTube видео"
        )

    async def handle_input(self, update: Update, context):
        message = await update.message.reply_text("🔄 Начало обработки...")
        try:
            # Определение типа ввода
            if update.message.audio:
                input_source = update.message.audio.file_id
            elif update.message.text:
                input_source = update.message.text
            else:
                await message.edit_text("❌ Неподдерживаемый формат")
                return

            # Обработка аудио/ссылки
            await message.edit_text("📥 Загрузка данных...")
            audio_path, source_type = await self.audio_processor.process_input(update, input_source)

            # Генерация протокола встречи
            await message.edit_text("🔍 Анализ содержимого...")
            protocol = await self.meeting_processor.process_meeting(audio_path)

            # Отправка результата
            prefix = "🎥 YouTube: " if source_type == "youtube" else "✅ Готово:"
            await message.edit_text(f"{prefix}\n\n{protocol}")

            # Очистка временного файла
            if os.path.exists(audio_path):
                os.remove(audio_path)

        except Exception as e:
            await message.edit_text(f"❌ Ошибка: {str(e)}")
            self.logger.error(f"Main Error: {str(e)}")

    def run(self):
        self.logger.info("Бот запущен")
        self.app.run_polling()

# ## Запуск бота

In [ ]:
# Точка входа в программу
if __name__ == "__main__":
    try:
        # Для корректной работы в Jupyter применяем nest_asyncio
        import nest_asyncio
        nest_asyncio.apply()

        print("🔑 Введите данные для настройки:")
        config = Config(
            TELEGRAM_TOKEN=getpass("Telegram Token (@BotFather): "),
            OPENAI_API_KEY=getpass("OpenAI API Key (https://platform.openai.com): ")
        )

        bot = NeuroSecretaryBot(config)
        bot.run()
    except Exception as e:
        print(f"Критическая ошибка: {str(e)}")